In [ ]:
%pip install langgraph

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")
from os import getenv

key = getenv("DEEPSEEK_API_KEY") 
url = getenv("DEEPSEEK_API_BASE")
model = getenv("DEEPSEEK_MODEL")

In [ ]:
from typing import Any, Dict, List, Optional, TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_deepseek import ChatDeepSeek

# 定义状态结构（必须继承TypedDict）
class DialogueState(TypedDict):
    messages: List[BaseMessage]  # 对话历史
    user_input: Optional[str]    # 当前输入的临时存储

def receive_user_input(state: DialogueState) -> DialogueState:
    """模拟接收用户输入"""
    user_input = input("> 你: ")
    print("> 你：", user_input)
    return {
        "messages": state["messages"] + [HumanMessage(content=user_input)], 
        "user_input": user_input
    }

def generate_ai_response(state: DialogueState) -> DialogueState:
    """生成 AI 回复"""
    chat_model = ChatDeepSeek(
        model=model,
        api_base=url,
        api_key=key
    )
    
    # 将历史消息转换为模型所需格式
    response = chat_model.invoke(state["messages"])
    
    # 更新状态
    print(f"> AI: {response.content}")
    return {
        "messages": state["messages"] + [AIMessage(content=response.content)], 
        "user_input": state["user_input"]
    }

# ---------- 流程构建 ----------
workflow = StateGraph(DialogueState)

# 添加节点
workflow.add_node("get_input", receive_user_input)  # 获取用户输入
workflow.add_node("generate_response", generate_ai_response)  # 生成回复

# 设置入口点
workflow.set_entry_point("get_input")

# ---------- 循环逻辑配置 ----------
def should_continue(state: DialogueState) -> str:
    """判断继续生成回复还是退出"""
    last_input = state["user_input"]
    
    # 退出条件：用户输入包含 'exit'
    if last_input and "exit" in last_input.lower():
        return END
    return "continue"   

workflow.add_conditional_edges(
    "get_input",
    should_continue,
    { "continue": "generate_response", END: END }  
)
workflow.add_edge("generate_response", "get_input")

# 编译为可执行应用
app = workflow.compile()

# ---------- 执行对话 ----------
print("对话开始（输入 'exit' 退出）...")
initial_state: DialogueState = {
    "messages": [],
    "user_input": None
}
    
final_state = app.invoke(initial_state)
